In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [3]:
import os
from langchain_tavily import TavilySearch

api_key = os.getenv("TAVILY_API_KEY")

tool = TavilySearch(
    tavily_api_key=api_key,
    max_results=5,
    topic="general",
    include_answer=False,
    include_raw_content=False,
    include_images=False,
    include_image_descriptions=False,
    search_depth="basic",
    time_range="month",
    include_domains=None,
    exclude_domains=None
)

tools = [tool]

In [4]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are a helpful AI assistant with access to web-search.

Use `TavilySearch` for: recent events, news, unknown facts, product info, research, anything that might require web search.
Answer directly without tools for: general knowledge, definitions, conversations, creative writing.

**EXAMPLES:**
User: "Who is the current president of the United States?" → Use TavilySearch
User: "What's the time right now?" → Use TavilySearch
User: "When was the first world war fought?" → Answer directly (general knowledge)
User: "Write a report on the GDP per capita of India in 1944" → Answer directly (general knowledge)
User: "Write a dialogue between Magnus Carlsen and MS Dhoni" → Answer directly (creative writing)
"""
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

In [5]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
query = input('Query : ')
result = agent_executor.invoke({'input': query})
print('AI: ', result)



> Entering new AgentExecutor chain...
Here's a dialogue between Magnus Carlsen, the world chess champion, and MS Dhoni, the former Indian cricket captain:

(Setting: A quiet café in Oslo, Norway. Magnus Carlsen and MS Dhoni sit across each other, sipping their coffee.)

Magnus Carlsen: Mahendra, it's an honor to finally meet you. I've heard so much about your incredible career in cricket.

MS Dhoni: Likewise, Magnus. Your achievements in chess are truly inspiring. I've always been fascinated by the strategic aspect of the game.

Magnus Carlsen: Yes, strategy is key in both chess and cricket, isn't it? The ability to think ahead, anticipate your opponent's moves, and adjust your plan accordingly.

MS Dhoni: Absolutely. In cricket, you need to read the game, understand the opposition's strengths and weaknesses, and make split-second decisions to outmaneuver them. It's similar to chess, where you need to analyze your opponent's moves and respond accordingly.

Magnus Carlsen: That's a gr